In [41]:
import gauleg as gl 
import sympy as sp 
import numpy as np 
import pandas as pd 
import math
import matplotlib.pyplot as plt
import Solver3 as sl 
from scipy.interpolate import interp1d

%load_ext autoreload

%autoreload 2


The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Generating Delta Vector

In [42]:
def interpolation_for_u_h(nodal, mesh, num_points):
  """
  Given the nodal solution 'nodal' computed on the refined mesh 'mesh',
  return the solution values at observation nodes computed as a uniform grid
  over [0,1] with num_points points.

  Assumes that every observation node exactly exists in the mesh.

  Parameters:
    nodal     : array-like, the computed solution at each mesh node.
    mesh      : array-like, the coordinates of the mesh nodes.
    num_points: int, number of observation points (e.g., 8).

  Returns:
    obs_values: np.array, the solution values corresponding to each observation node.

  Raises:
    ValueError if any observation node is not found in the mesh.
  """
  nodal = sl.assemble_nodal_values(nodal)
  mesh = np.asarray(mesh)
  nodal = np.asarray(nodal)
  
  # Compute observation nodes uniformly over [0,1]
  obs_nodes = np.linspace(0.0, 1.0, num_points)
  obs_values = np.zeros(num_points)

  for i, x_obs in enumerate(obs_nodes):
    idx = np.where(mesh == x_obs)[0]
    if idx.size == 0:
      raise ValueError(f"Observation node {x_obs} not found in the mesh.")
    obs_values[i] = nodal[idx[0]]

  return obs_values


def cov_matrix(sigma, num_points):
  sigma = (sigma ** 2) * np.eye(num_points)
  return sigma


def compute_basis_functions_at_x(x, mesh):
    """
    Given an observation point x and the mesh, compute the two nonzero
    finite element basis (hat) functions at x.
    
    Returns:
      k: the index such that x is in [mesh[k], mesh[k+1]]
      psi_left: the value of the basis function associated with mesh[k] at x.
      psi_right: the value of the basis function associated with mesh[k+1] at x.
    """
    mesh = np.asarray(mesh, dtype=float)
    # Find the index k such that x is in [mesh[k], mesh[k+1]]
    k = np.searchsorted(mesh, x) - 1
    k = np.clip(k, 0, len(mesh) - 2)  # ensure k is valid
    
    psi_left = sl.phi_i(k, x, mesh)
    psi_right = sl.phi_i(k+1, x, mesh)
    return k, psi_left, psi_right

# Example: Evaluate the FE solution at a set of observation points
def fe_solution_at_obs(c_sol, mesh, x_obs):
    """
    Compute the finite element solution at observation points x_obs,
    given the nodal solution c_sol and the mesh.
    
    Parameters:
      c_sol : array of nodal values (length N)
      mesh  : array of node coordinates (length N)
      x_obs : array of observation points
      
    Returns:
      c_interp: array of interpolated FE solution values at x_obs.
    """
    c_sol = sl.assemble_nodal_values(c_sol)
    c_sol = np.asarray(c_sol).flatten()
    c_interp = np.zeros_like(x_obs, dtype=float)
    
    for idx, x in enumerate(x_obs):
        k, psi_left, psi_right = compute_basis_functions_at_x(x, mesh)
        # The solution at x is the weighted average of the two nodal values
        c_interp[idx] = c_sol[k] * psi_left + c_sol[k+1] * psi_right
    return c_interp

# Alternatively, if many observation points fall in the same element,
# you could vectorize the process by grouping x_obs by element.



In [43]:
def add_noise(observations_at_xi, num_points, sigma):
    """
    Adds a normally distributed noise, theta
    to observations from the forward solver.

    Arguments:
    observations_at_xi : observations at predetermined xi using interplotion. 
    num_points : how big your covariance matrix is 

    Returns:
    Delta : Array of Noisy observations.
    
    """
    sigma = cov_matrix(sigma, num_points)
    noise = np.random.multivariate_normal(np.zeros(num_points), sigma)
    delta = observations_at_xi + noise 
    return delta



# Define Likelihood Function 

In [44]:
def phi(observations, predicted, sigma, num_points) :
    '''
    For a set of predetermined points xi -- obtained via np.linspace,
    this function defines the likelihood function 

    Arguments:
    observations: Generated noisy observation using beta_true -- corresponds to y in literature
    predicted: For a proposed beta_i, we compute the noisy observation using the forward solver 
    -- corresponds to g(beta_i) in literature

    Returns: 
    Likelihood function that is proportional to the prior distribution
    
    '''
    covariance_matrix = cov_matrix(sigma, num_points)
    diff = predicted - observations
    covariance_matrix_inv = np.linalg.inv(covariance_matrix) 
    val = 0.5 * diff.T @ covariance_matrix_inv @ diff
    return val



In [45]:
def compute_A(phi_0, phi_i, sigma):

    val = np.exp(phi_0 - phi_i)
    
    return val

# MCMC algorithm 

In [55]:
def MCMC(beta_true, number_of_iter, burn_in, sigma, num_points): 
    '''
    Builds a Markov Chain 

    Key Steps:
    1. Initialise a choice of Beta, beta_0 
    2. Compute likelihood of beta_0, using delta and beta_0_predicted
    3. Initialise the loop.
        - we propose a new beta_i from x* ~ Uniform(0.15, 0.85) and r ~ Uniform(0, 0.15)
        - compute y_i and g(beta_i)
        - compute likelihood using {y_i and g(beta_i)}
        - set alpha = min{1, likelihood }
    '''
    # set seed 
    np.random.seed(42)
    # range of uniform distribution 
    x_star_range = (0.3, 0.7) 
    r_range = (0.1, 0.2) 
    chain = []
    ddof_list = []
    # compute delta 
    mesh_true , c_sol_true, ddof_true= sl.refinement_loop(0.0001, beta_true) 
    y_true = fe_solution_at_obs(c_sol_true, mesh_true, np.linspace(0.0, 1.0, num_points))
    delta = add_noise(y_true, num_points, sigma)

    # draw first copy of beta --> beta_0

    beta_0 = np.array([
            np.random.uniform(*x_star_range),
            np.random.uniform(*r_range)
        ])
    print("Beta_0:", beta_0)
    
    # initialise current observations and likelihood 
    mesh_0, c_sol_0, ddof_0 = sl.refinement_loop(0.01, beta = beta_0)
    y_0 = fe_solution_at_obs(c_sol_0, mesh_0, np.linspace(0.0, 1.0, num_points))
    phi_0 = phi(delta, y_0, sigma, num_points)
    print("phi_0:", phi_0)
    ddof_list.append((beta_0, ddof_0))
    
    iter_count = 0
    acceptance_count = 0 
    acceptance_prob_history = []

    for i in range(number_of_iter):
        beta_proposal = np.array([
            np.random.uniform(*x_star_range),
            np.random.uniform(*r_range)
        ])
        print("beta proposal:", beta_proposal)
        mesh_proposal, c_sol_proposal, ddof_proposal = sl.refinement_loop(0.01, beta = beta_proposal)
        y_proposal = fe_solution_at_obs(c_sol_proposal, mesh_proposal, np.linspace(0.0, 1.0, num_points))
        ddof_list.append((beta_proposal, ddof_proposal))
        phi_proposal = phi(delta, y_proposal, sigma, num_points)
        print("phi proposal:", phi_proposal)
        # compute acceptance probability 
        A = compute_A(phi_0, phi_proposal, sigma)
        acceptance_prob = min(1, A)
        acceptance_prob_history.append(acceptance_prob)
        print("acceptance probablity:", acceptance_prob)
        
        # Accept or reject the proposal
        if np.random.rand() < acceptance_prob:
            beta_0 = beta_proposal # update the current state as the last accepted proposal
            y_0 = y_proposal # update the current observations to the last accepted observation
            phi_0 = phi_proposal
            acceptance_count += 1
        
        # Record the current state.
        chain.append(beta_0.copy())
        print("Chain length:", len(chain))
        
    
    chain = np.array(chain)
    # Compute the MCMC estimate as the mean of the samples after burn-in.
    beta_mcmc = np.mean(chain[burn_in:], axis=0)
    
    return chain, beta_mcmc, acceptance_prob_history, acceptance_count, ddof_list

In [56]:
number_of_iter = 10000
burn_in = 5000
sigma = 0.01
num_points = 100
beta_true = np.array([0.65, 0.15])

chain, beta_mcmc, acceptance_history, acceptance_count, ddof_list = MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmc)
print("Acceptance Probability History:", acceptance_history)
print("Acceptance Count:", acceptance_count)
print("ddof list:", ddof_list)

Beta_0: [0.4669644  0.12221078]
phi_0: 65.95117834303096
beta proposal: [0.34794615 0.13376152]
phi proposal: 59.097664971896634
acceptance probablity: 1
Chain length: 1
beta proposal: [0.42928117 0.15187906]
phi proposal: 62.333222942179596
acceptance probablity: 0.039338249241301644
Chain length: 2
beta proposal: [0.44545184 0.19717821]
phi proposal: 54.88447943276228
acceptance probablity: 1
Chain length: 3
beta proposal: [0.40071292 0.14972485]
phi proposal: 61.83448106710795
acceptance probablity: 0.0009586335869540831
Chain length: 4
beta proposal: [0.4139362  0.10368869]
phi proposal: 66.97119998617133
acceptance probablity: 5.633832870438222e-06
Chain length: 5
beta proposal: [0.50107161 0.10514788]
phi proposal: 64.92354273808489
acceptance probablity: 4.366065060520468e-05
Chain length: 6
beta proposal: [0.66330635 0.12395619]
phi proposal: 40.00225588793694
acceptance probablity: 1
Chain length: 7
beta proposal: [0.4957811  0.19856505]
phi proposal: 48.98463930173531
accepta

In [ ]:
mean_beta = np.mean(chain, axis=0)
mean_beta_sq = np.mean(chain**2, axis=0)
var_beta = mean_beta_sq - mean_beta**2
var_beta

array([3.43004177e-03, 4.12854339e-05])

# DataLogging : 
```
1. Chain Length of 10000 with 1000 burn in takes ~~ 900 mins --> 15 hours 
   Params and Results: 
    number_of_iter = 10000 
    burn_in = 1000 
    sigma = 0.05 
    num_points = 8 
    beta_true = np.array([0.23, 0.13]) 
    MCMC estimated beta: [0.28158902 0.06286484]
    variance(beta) = array([0.00919061, 0.00088973])
Comments: Phi and Sigma computation is wrong, the area of acceptance is too small 
Run Profilier 
Heat map
Check computation of Phi and A 
fix right hand side at 1 
set epislon to be 0.01

2. Chain length of 10000 with 5000 burn in takes ~~ 22 mins
    Params and Results: 
    number_of_iter = 10000
    burn_in = 5000
    sigma = 0.01
    num_points = 100
    beta_true = np.array([0.65, 0.15])
    MCMC estimated beta: [0.64309708 0.16234643]
    Acceptance Count: 917
    
```

# Heat Map of Beta potential 

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import plotly.graph_objects as go

# Parameters for the parameter grid (x* and r)
x_star_vals = np.linspace(0.3, 0.7, 20)   # More candidate x* values for finer resolution
r_vals = np.linspace(0.1, 0.2, 5)          # More candidate r values

# Create a mesh grid of parameter pairs
X, R = np.meshgrid(x_star_vals, r_vals)

# Global parameters (use the same parameters throughout)
epsilon = 1e-3       # Refinement tolerance
sigma = 0.01         # Noise standard deviation
num_points = 100     # Number of observation points

# Generate the "true" observation using beta_true
beta_true = np.array([0.65, 0.15])
mesh_true, c_sol_true = sl.refinement_loop(epsilon, beta_true, num_points)
y_true = interpolation_for_u_h(c_sol_true, mesh_true, num_points)
observations = y_true + add_noise(y_true, num_points, sigma)
true_potential = phi(observations, y_true, sigma, num_points)

# Prepare an array to store the misfit potential values
potential = np.zeros_like(X)

# Loop over the grid of (x*, r) candidates
for i in range(X.shape[0]):
    for j in range(X.shape[1]):
        beta_candidate = np.array([X[i, j], R[i, j]])
        # Solve the forward problem for the candidate beta
        mesh_candidate, c_sol_candidate = sl.refinement_loop(epsilon, beta_candidate, num_points)
        y_candidate = interpolation_for_u_h(c_sol_candidate, mesh_candidate, num_points=num_points)
        # Compute the misfit potential (phi)
        potential[i, j] = phi(observations, y_candidate, sigma, num_points)
        
# Create a 3D surface plot of the misfit potential
fig = go.Figure(data=[go.Surface(x = X, y = R, z = potential, colorscale = 'turbid'),
                      go.Scatter3d(
        x=[beta_true[0]], 
        y=[beta_true[1]], 
        z=[true_potential],
        mode='markers',
        marker=dict(size=10, color='cyan'),
        name='True beta'
    )])

# Update layout with axis titles
fig.update_layout(
    title='Interactive 3D Surface Plot',
    scene=dict(
        xaxis_title='x*',
        yaxis_title='r',
        zaxis_title='Potential'
    ),
    autosize = True,
    width = 500,
    height = 500 
)

fig.show()


C:\Users\Admin\AppData\Local\Temp\ipykernel_10204\4160746210.py:32: DeprecationWarning:

Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)



KeyboardInterrupt: 

In [ ]:
beta_true 

In [ ]:
from concurrent.futures import ProcessPoolExecutor

def run_single_chain(seed, beta_true, number_of_iter, burn_in, sigma, num_points):
    # Set the random seed to ensure independent chains.
    np.random.seed(seed)
    return MCMC(beta_true, number_of_iter, burn_in, sigma, num_points)

def run_multiple_chains(n_chains, beta_true, number_of_iter, burn_in, sigma, num_points):
    seeds = np.random.randint(0, 10000, size=n_chains)
    chains = []
    beta_mcmcs = []
    acceptance_histories = []
    acceptance_counts = []
    
    # Run chains in parallel using ProcessPoolExecutor
    with ProcessPoolExecutor() as executor:
        # Submit all the chains concurrently
        futures = [
            executor.submit(run_single_chain, seed, beta_true, number_of_iter, burn_in, sigma, num_points)
            for seed in seeds
        ]
        # Collect results as they complete
        for future in futures:
            chain, beta_mcmc, acceptance_prob_history, acceptance_count = future.result()
            chains.append(chain)
            beta_mcmcs.append(beta_mcmc)
            acceptance_histories.append(acceptance_prob_history)
            acceptance_counts.append(acceptance_count)
    
    return chains, beta_mcmcs, acceptance_histories, acceptance_counts

if __name__ == '__main__':
    n_chains = 4
    number_of_iter = 10000
    burn_in = 1000
    sigma = 0.1 
    num_points = 100
    beta_true = np.array([0.5, 1/6])
    chains, beta_mcmcs, acceptance_histories, acceptance_counts = run_multiple_chains(
        n_chains, beta_true, number_of_iter, burn_in, sigma, num_points
    )
print("True beta:", beta_true )
print("MCMC estimated beta:", beta_mcmcs)
print("Acceptance Probability History:", acceptance_histories)
print("Acceptance Count:", acceptance_counts)